In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json('/media/sidhu/Samsung 1TB/sysmon.json', lines=True, chunksize=100000)

This is a hackish loop to convert my sysmon data that I exported from elastic.
It removes elastic metafield data, adds some more structured data that was previous

In [ ]:
for ind, chunk in enumerate(df):
    chunk.drop(['_index', '_type', '_id', '_score'], axis=1, inplace=True)
    json_df = pd.io.json.json_normalize(chunk['_source'].tolist())
    
    # Add message column information to the rest of the data
    dict_data = []
    for items in json_df['message'].tolist():
        temp_dict = {}
        for item in str(items).split('\n'):
            split = item.split(':')
            
            # Since the image contains the full path of the process
            # It gets split on ":" so we add a condition to handle it
            # Ignore the message "Network Connection Detected"
            if len(split) >= 2:
                if split[0] == 'Image' and len(split) > 2:
                    temp_dict[split[0]] = split[1] + ':' + split[2]
                elif split[0].endswith('detected'):
                    pass
                else:
                    temp_dict[split[0]] = split[1].strip()
        dict_data.append(temp_dict)

    message_data = pd.io.json.json_normalize(dict_data)
    full_data = pd.concat([json_df, message_data], axis=1).drop('message', axis=1)
    json_df.to_csv(f'/home/sidhu/data_files/sysmon-{ind}.csv')

    if ind % 10 == 0:
        print(ind)

0
10
20
